In [2]:
# encoding = 'utf-8'

import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression  
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [ ]:
df = pd.read_csv('no_missing_data.csv', header=0, encoding='utf-8')

df

In [5]:
df.shape

(4833, 52)

In [6]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

# Indices of the two rows to swap
index1, index2 = 14, 0

# Perform the swap
df.loc[index1], df.loc[index2] = df.loc[index2].copy(), df.loc[index1].copy()

# Drop columns that are not needed
columns_to_drop = ['StudentNumber', 'useraccount_id', 'SchoolName', 'AlgScaleScore']
df = df.drop(columns=columns_to_drop, axis=1)

# Define variable types
binary_variables = ['Sex', 'FRL_Status', 'Race_Indicator', 'Hispanic_Indicator', 'RetakerFlag']
nominal_variables = ['SchoolNumber']
continuous_variables = ['FSAMath_2018_AchievementLevel', 'TotalNumberofAbsences', 
                        'sum_session', 'video_id', 'total_video_time', 'video_completed', 'video_pause', 'video_play', 'video_seek', 'avg_user_gave_correct_answer', 
                        'tys_finish', 'tys_previous', 'tys_review_incorrect_question', 'tys_review_solution_video', 'wall_page_load']

# Create a preprocessor for the continuous variables
continuous_preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values using the mean
    ('scaler', StandardScaler())  # Standardize features by removing the mean and scaling to unit variance
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', continuous_preprocessor, continuous_variables),
        # ('cat', OneHotEncoder(handle_unknown='ignore'), nominal_variables)  # Uncomment and adjust if categorical processing is needed
    ])

# Define a logistic regression model
logistic_model = LogisticRegression(max_iter=1000)  # Increase the number of iterations to ensure convergence

# Create a complete pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Apply the preprocessing steps
    ('classifier', logistic_model)  # Use logistic regression as the classifier
])

# Select columns to keep for the model input
columns_to_keep = binary_variables + nominal_variables + continuous_variables

# Filter the DataFrame to only the necessary columns
X = df[columns_to_keep]

# Assume 'risk' is the target variable
y = df['risk']


In [ ]:
X

In [ ]:
y

In [ ]:
df[columns_to_keep + ['risk']]

### LR model

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 设置随机种子以确保结果可复现
torch.manual_seed(0)

In [34]:
data = df[columns_to_keep + ['risk']]
data.shape

(4833, 25)

In [35]:
data.head()

,RetakerFlag,FRL_Status,Sex,Race_Indicator,Hispanic_Indicator,SchoolNumber,TotalNumberofAbsences,FSAMath_2018_AchievementLevel,FSAMath_2018_ScaleScore,sum_session,...,video_pause,video_play,video_seek,avg_user_gave_correct_answer,tys_finish,tys_previous,tys_review_incorrect_question,tys_review_solution_video,wall_page_load,risk
0,0,1,1,1.0,1,251,5,2.0,328.0,20.0,...,298.0,272.0,608.0,0.10,69.0,14.0,92.0,1.0,54.0,1
1,0,1,0,1.0,0,1681,2,3.0,349.0,10.0,...,60.0,58.0,120.0,0.10,0.0,0.0,0.0,0.0,3.0,1
2,0,0,1,1.0,1,3971,5,4.0,359.0,10.0,...,352.0,294.0,3878.0,0.70,0.0,0.0,0.0,0.0,16.0,1
3,0,1,0,1.0,0,171,7,1.0,307.0,20.0,...,176.0,173.0,72.0,0.10,3.0,7.0,5.0,0.0,43.0,1
4,0,1,0,1.0,0,1451,8,3.0,346.0,40.0,...,331.0,378.0,335.0,0.35,11.0,0.0,9.0,20.0,40.0,1


In [39]:
X = data.drop('risk',1)
y = data[['risk']]

X = torch.tensor(np.array(X)).float()
y = torch.tensor(np.array(y).flatten()).float()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Splitting the dataset
train_size = int(num_samples * 0.8)  # 80% of the dataset is used for training
test_size = num_samples - train_size  # Remaining 20% is used for testing
train_dataset = TensorDataset(X[:train_size], y[:train_size])  # Creating the training dataset
test_dataset = TensorDataset(X[train_size:], y[train_size:])  # Creating the testing dataset
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # DataLoader for the training dataset
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)  # DataLoader for the testing dataset

# Define a logistic regression model
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        # Linear layer that outputs to one unit, expecting sigmoid activation for binary classification
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        # Applying the sigmoid function to the output of the linear layer
        outputs = torch.sigmoid(self.linear(x))
        return outputs

# Initialize the model with the number of features
model = LogisticRegressionModel(num_features)

# Loss function and optimizer
criterion = nn.BCELoss()  # Binary cross-entropy loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001

# Train the model
num_epochs = 10  # Number of training epochs
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Clear gradients for this training step
        outputs = model(inputs)  # Forward pass to get output
        loss = criterion(outputs, labels.view(-1, 1))  # Calculate loss
        loss.backward()  # Getting gradients w.r.t. parameters
        optimizer.step()  # Updating parameters

# Evaluate the model
model.eval()  # Sets the model to evaluation mode
with torch.no_grad():  # Gradient computation is disabled since the model is being evaluated
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)  # Compute model output
        predicted = (outputs.data > 0.5).float()  # Convert output probabilities to binary predictions
        total += labels.size(0)  # Total number of labels
        correct += (predicted.view(-1) == labels).sum().item()  # Count correct predictions

# Calculate and print accuracy
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')  # Print the accuracy in percentage


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Set the model to evaluation mode
model.eval()
y_true = []  # List to store actual labels
y_pred = []  # List to store predicted labels

# Disable gradient computation to speed up the process and reduce memory usage
with torch.no_grad():
    for inputs, labels in test_loader:  # Iterate through the test dataset
        outputs = model(inputs)  # Forward pass: compute the output of the model
        predicted = (outputs.data > 0.5).float().view(-1)  # Apply threshold to get binary predictions
        y_true.extend(labels.tolist())  # Extend the list of actual labels
        y_pred.extend(predicted.tolist())  # Extend the list of predictions

# Compute evaluation metrics
accuracy = accuracy_score(y_true, y_pred)  # Calculate the accuracy
precision = precision_score(y_true, y_pred)  # Calculate the precision
recall = recall_score(y_true, y_pred)  # Calculate the recall
f1 = f1_score(y_true, y_pred)  # Calculate the F1 score

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Print the metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

### LR with TE constrain

In [49]:
# ['FRL_Status', 'Sex', 'Race_Indicator', 'Hispanic_Indicator'] 这几个特征在的位置为1,2,3,4,所以索引列表为[1,2,3,4]
sensitive_feature_indices = [1,2,3,4]

In [46]:
# Build a logistic regression model
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        # Define a linear layer with input dimension to 1 output (binary classification)
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        # Apply the sigmoid activation function to the linear layer's output
        outputs = torch.sigmoid(self.linear(x))
        return outputs

# Instantiate the model with the number of features
model = LogisticRegressionModel(num_features)

# Define a custom loss function
def custom_loss(outputs, labels, inputs, lambda_fairness=0.01):
    # Basic loss: binary cross-entropy
    criterion = nn.BCELoss()
    loss = criterion(outputs, labels.view(-1, 1))

    # Add fairness loss
    fairness_loss = 0
    # Calculate fairness metric for each sensitive feature index
    for idx in sensitive_feature_indices:
        fairness_loss += calculate_group_fairness_metric(outputs, labels, inputs[:, idx])
    # Total loss includes both basic and fairness-induced losses
    total_loss = loss + lambda_fairness * fairness_loss

    return total_loss

def calculate_group_fairness_metric(outputs, labels, sensitive_feature):
    epsilon = 1e-6  # A small constant to prevent division by zero

    # Split data into two groups based on sensitive feature
    group_0 = (sensitive_feature == 0)
    group_1 = (sensitive_feature == 1)

    # Compute False Positive Rate (FPR) and False Negative Rate (FNR) for each group
    fpr_0 = torch.mean(outputs[group_0 & (labels == 0)]) if group_0.any() and (labels[group_0] == 0).any() else torch.tensor(0.0)
    fnr_0 = torch.mean(1 - outputs[group_0 & (labels == 1)]) if group_0.any() and (labels[group_0] == 1).any() else torch.tensor(0.0)
    fpr_1 = torch.mean(outputs[group_1 & (labels == 0)]) if group_1.any() and (labels[group_1] == 0).any() else torch.tensor(0.0)
    fnr_1 = torch.mean(1 - outputs[group_1 & (labels == 1)]) if group_1.any() and (labels[group_1] == 1).any() else torch.tensor(0.0)

    # Calculate a fairness metric based on FPR and FNR
    return torch.abs(fnr_0 * fpr_1/(fpr_0 * fnr_1 + epsilon)-1)

# Set up the optimizer with Adam algorithm and a learning rate of 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        # Reset gradients
        optimizer.zero_grad()
        # Forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(inputs)
        # Calculate loss
        loss = custom_loss(outputs, labels, inputs)
        # Backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # Perform a single optimization step (parameter update)
        optimizer.step()

In [ ]:
# Model evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs.data > 0.5).float()
        total += labels.size(0)
        correct += (predicted.view(-1) == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')

In [ ]:
import numpy as np

def calculate_treatment_equality(y_true, y_pred, groups):
    epsilon = 1e-6
    treatment_equality_diff = []

    # Assuming that 'groups' contains the same keys as in the original example: 'FRL_Status', 'Sex', 'Race_Indicator', 'Hispanic_Indicator'
    for g in np.unique(groups):
        # Filter the predictions and labels for each group
        idx_group_0 = groups == 0
        idx_group_1 = groups == 1

        y_true_g0 = y_true[idx_group_0]
        y_pred_g0 = y_pred[idx_group_0]
        y_true_g1 = y_true[idx_group_1]
        y_pred_g1 = y_pred[idx_group_1]

        # Compute False Negative Rate (FNR) and False Positive Rate (FPR)
        fnr_group_0 = np.mean((y_pred_g0 == 0) & (y_true_g0 == 1))
        fpr_group_0 = np.mean((y_pred_g0 == 1) & (y_true_g0 == 0))
        fnr_group_1 = np.mean((y_pred_g1 == 0) & (y_true_g1 == 1))
        fpr_group_1 = np.mean((y_pred_g1 == 1) & (y_true_g1 == 0))

        # Compute the treatment equality difference using the specified formula
        diff = abs(fnr_group_0 * fnr_group_1 / (fpr_group_0 * fpr_group_1 + epsilon))
        treatment_equality_diff.append(diff)

    # Calculate the average difference across all features
    overall_te = np.mean(treatment_equality_diff) if treatment_equality_diff else 0
    return overall_te

# Example usage in a PyTorch evaluation context
model.eval()
y_true = []
y_pred = []
groups = []

with torch.no_grad():
    for inputs, labels, group in test_loader:  # Assume group information is also loaded
        outputs = model(inputs)
        predicted = (outputs.data > 0.5).float().view(-1)
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())
        groups.extend(group.numpy())

# Convert lists to numpy arrays for easier manipulation
y_true = np.array(y_true)
y_pred = np.array(y_pred)
groups = np.array(groups)

# Calculate Treatment Equality
treatment_equality = calculate_treatment_equality(y_true, y_pred, groups)
print(f'Treatment Equality: {treatment_equality}')
